In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, scipy as sci, csv
from numpy import exp, pi, sinc, sin, tan, convolve, square, abs, arcsin, arctan, sqrt, cos, shape, array, mean
from scipy.signal import unit_impulse as dirac, find_peaks
period='holder'

In [ ]:
class Diffraction_pattern:
    def __init__(self, lamda, height, inc_angle):
        self.lamda = lamda
        self.height = height
        self.inc_angle = inc_angle/180*pi
        self.frequency = np.arange(sin(-90/180*pi)/self.lamda,sin(90/180*pi)/self.lamda,100)
        self.blaze_angle = None
        self.angles_rad = arcsin(self.frequency*self.lamda)#[arcsin(x) for x in self.frequency*self.lamda]
        self.angles_deg = self.angles_rad/pi*180 #[angle_rad/pi*180 for angle_rad in self.angles_rad]
        self.period = None
        self.tick_height = 0
        self.length = 100*1e-6 # size of the irradiated part of the grating in nanometers
        self.n = 1.6
        for T in range(30,200,10):
            self.period = T*1e-7
            self.transmissive()
            self.reflective()
        
        
    def sha(self):    # operational
        pulse, _ = np.array(find_peaks(-np.abs(sin((self.frequency)*2*pi/(2/self.period))), distance= 200))
        comb = np.zeros(len(self.frequency - sin(self.inc_angle)/self.lamda))
        comb[pulse] = 1
        return 1/self.period * comb
        
    def aperture(self):
        return self.length * sinc(self.frequency*self.length/2)
    
    def Rphasebox(self):
        irradiated = cos(self.inc_angle)/cos(self.blaze_angle-self.inc_angle) #* pi*self.period
        offset = irradiated*(sin(self.inc_angle - self.blaze_angle) + sin(- self.blaze_angle))/self.lamda
        return self.period * sinc((self.frequency + offset)*self.period/2)
    
    def Rphasebox_wave(self):
        irradiated = cos(self.inc_angle)/cos(self.blaze_angle-self.inc_angle) #* pi*self.period
        offset = irradiated*(sin(self.inc_angle - self.blaze_angle) + sin(self.angles_rad - self.blaze_angle))/self.lamda
        return self.period * sinc((self.frequency - offset)*self.period/2)
    
    def Tphasebox_complicated(self):
        irradiated = cos(self.inc_angle)/cos(self.blaze_angle-self.inc_angle)
        incoming = irradiated*(sin(self.inc_angle - self.blaze_angle))
        through = tan(self.blaze_angle)/cos(self.blaze_angle - arcsin(sin(self.inc_angle + self.blaze_angle)/self.n))
        out = sin(arcsin(sin(self.inc_angle+self.blaze_angle)/self.n)+self.blaze_angle)
        offset = (incoming - through)/self.lamda
        return self.period * sinc((self.frequency + offset)*self.period/2) #* #exp(-1j*2*pi/self.lamda*tan(self.blaze_angle)*self.period)
        
    def Tphasebox(self):
        offset = (- tan(self.blaze_angle) + self.n*tan(self.blaze_angle) + sin(self.inc_angle))/self.lamda
        return self.period * sinc((self.frequency - offset)*self.period/2)
    
    def Rmax(self):
        ref_angle = -self.inc_angle + 2*self.blaze_angle
        ref_angle = ref_angle*180/pi
        return ref_angle
    
    def Tmax(self):
        trans_angle = arcsin(self.n*sin(self.blaze_angle - arcsin(sin(self.inc_angle+self.blaze_angle)/self.n)))
        trans_angle = trans_angle*180/pi
        return trans_angle
                        
    def Rtotal(self):
        result = convolve(self.Rphasebox() * self.sha(), self.aperture(), mode = 'same')
        return result

    def Ttotal(self):
        result = convolve(self.Tphasebox() * self.sha(), self.aperture(), mode = 'same')
        return result
    
    def peaks(self):   # operational
        peaks = [arcsin(sin(self.inc_angle) - m*self.lamda/self.period) for m in range(-30,30)]
        return [peak/pi*180 for peak in peaks if abs(peak)<1.5]
    
    def transmissive(self, show_orders = True, show_max = True):
            self.blaze_angle = arctan(self.height/self.period)
            y = square(abs(self.Tphasebox()))
            y_total = square(abs(self.Ttotal()))
            y_complicated = square(abs(self.Tphasebox_complicated()))
            self.tick_height = max(y_total)/2
            peaks = self.peaks()
            maxima = self.Tmax()
            
            fig = plt.figure(figsize=(30,6))
            ax = fig.add_subplot(1, 1, 1)
            plt.title("Transmittance", fontsize = 24)
            plt.text(0.3, 0.6, 'Period: '+str(round(self.period,8))+'m \nBlaze angle: '+str(round(self.blaze_angle/pi*180,1))+' \nIncident angle: '+str(round(self.inc_angle/pi*180,1)),transform = ax.transAxes, fontsize =24)
            plt.xlabel('Difraction angle[°]')
            plt.ylabel('Intesity')
            plt.plot(self.angles_deg, y/max(y)*max(y_total), label = 'phase term')
            plt.plot(self.angles_deg, y_total, label = 'total transmittance')
            if show_orders: plt.vlines(peaks, np.zeros(len(peaks)), np.full(len(peaks),self.tick_height/2), color = 'black', alpha = 0.2, label = 'peak positions')
            if show_max: plt.vlines(maxima, 0, self.tick_height, color = 'green')
            plt.legend(fontsize=24)
            plt.show()
            
    def reflective(self, show_orders = True, show_max = True):
            self.blaze_angle = arctan(self.height/self.period)
            y = square(abs(self.Rphasebox()))
            y_wave = square(abs(self.Rphasebox_wave()))
            y_total = square(abs(self.Rtotal()))
            self.tick_height = max(y_total)/2
            peaks = self.peaks()
            maxima = self.Rmax()
            
            fig = plt.figure(figsize=(30,6))
            ax = fig.add_subplot(1, 1, 1)
            plt.title("Reflective phase - without refracted angle", fontsize = 24)
            plt.text(0.3, 0.6, 'Period: '+str(round(self.period,8))+'m \nBlaze angle: '+str(round(self.blaze_angle/pi*180,1))+' \nIncident angle: '+str(round(self.inc_angle/pi*180,1)),transform = ax.transAxes, fontsize =24)
            plt.xlabel('Difraction angle[°]')
            plt.ylabel('Intesity')
            plt.plot(self.angles_deg, y/max(y)*max(y_total), label = 'phase term')
            plt.plot(self.angles_deg, y_total, label = 'total reflectance')
            if show_orders: plt.vlines(peaks, np.zeros(len(peaks)), np.full(len(peaks),self.tick_height*2), color = 'black', alpha = 0.2, label = 'peak positions')
            if show_max: plt.vlines(maxima, 0, self.tick_height, color = 'green', label = 'geometrical optics maxima')
            plt.legend(fontsize=24)
            plt.show()
    
dif = Diffraction_pattern(lamda = 0.5e-6, height = 4e-6, inc_angle = 0)

In [ ]:
def delta(frequency, lamda,inc_ang):
    offset = sin(inc_ang/180*pi)/lamda
    pulse, _ = np.array(find_peaks(-abs(frequency-offset), distance= 200))
    delta = np.zeros(len(frequency))
    delta[pulse] = 1
    return delta

def sha(frequency, lamda):
    pulse, _ = np.array(find_peaks(-np.abs(sin(frequency*2*pi/(2/period))), distance= 200))
    comb = np.zeros(len(frequency))
    comb[pulse] = 1
    return 1/period * comb

def boxf(frequency, lamda):
    return period * sinc(frequency*period/2)

def bigboxf(frequency, lamda, L=5*period, inc_ang=0):
    L= L*period
    offset = sin(inc_ang/180*pi)/lamda
    return L * sinc((frequency-offset)*L/2)

def transphase(frequency, lamda,n):
    offset = (- tan(blaze_angle) + n*tan(blaze_angle))/lamda
    return period * sinc((frequency-offset)*period/2) * exp(-1j*2*pi/lamda*tan(blaze_angle)*period)

def transmissive(frequency,n, lamda,inc_angle):
    separate = square(abs(transphase(frequency, lamda, n)))
    """
    if len(separate) == len(angles_deg[0]):
        fig = plt.figure(figsize=(30,6))
        plt.plot(angles_deg[0],separate)
        plt.show()
    """
    result = convolve(sha(frequency, lamda)* transphase(frequency, lamda, n),bigboxf(frequency, lamda,L=50),mode='same')
    result = convolve(result,delta(frequency, lamda,inc_ang = inc_angle), mode = 'same')
    return square(abs(result))

def reflectphase(frequency, lamda,n):
    offset = (sin(ref_angle) - tan(inc_angle - blaze_angle))/cos(blaze_angle)/lamda
    return period * sinc((frequency-offset)*period/2)

def reflective(frequency,n, lamda,inc_angle):
    separate = square(abs(reflectphase(frequency, lamda, n)))
    """
    if len(separate) == len(angles_deg[0]):
        fig = plt.figure(figsize=(30,6))
        plt.plot(angles_deg[0],separate)
        plt.show()
    """
    result = convolve(sha(frequency, lamda)* reflectphase(frequency, lamda, n),bigboxf(frequency, lamda,L=50),mode='same')
    return square(abs(result))

# Just one slit with variable incoming angle
$$\Large \mathscr{F}(rect(\frac{x}{d}) \cdot exp(-ikd)) = \mathscr{F}(rect(\frac{x}{d}) \cdot exp(-i\frac{2\pi}{\lambda}sin(\theta)x))$$
$$\Large I = |\mathscr{F}(rect(\frac{x}{d}) \cdot exp(-ixsin(\alpha)|^2 = |d \cdot sinc((\nu - sin(\alpha)))|^2$$
$$\Large \theta_m = \arcsin(sin(\theta_{inc})-m\frac{\lambda}{d})$$
$$\Large I = |\mathscr{F}(rect(\frac{x}{d}) \ast Ш_d \cdot exp(-ixsin(\alpha))|^2 = |d \cdot sinc(\nu d) \cdot \frac{1}{d} \cdot Ш_{1/d} \ast \delta(\nu + \frac{sin(\alpha)}{2\pi})|^2 = |sinc(\nu d)\cdot Ш_{1/d} \ast \delta(\nu - \frac{\sin(\alpha)}{2\pi})|^2$$

# Just one prism

 phase diffrence: $$ \Large exp(-iksin(\alpha)x) \cdot exp(-ik(dtan(\beta)-xtan(\beta)) \cdot exp(-inkxtan(\beta))$$
 
 $$ \Large exp(-ika) \cdot exp(-ikx(sin(\alpha) -tan(\beta) +ntan(\beta))$$
 
 $$\Large I = |\mathscr{F}(rect(\frac{x}{d}) \cdot exp(-ika) \cdot exp(-ikx(sin(\alpha) -tan(\beta) +ntan(\beta))|^2 $$
 
 $$\Large I = |d \cdot sinc(\nu d - offset)|^2$$
 

In [ ]:
lamda = array([0.47e-6,0.53e-6,0.68e-6]) # wavelength representing blue colo
selm_lamda = lamda*10e6
n = array([[sqrt(1.318 + 1.194*lam**2/(lam**2-0.17104**2))] for lam in selm_lamda])
frequency = array([np.arange(sin(-90/180*pi)/lam,sin(90/180*pi)/lam,100) for lam in lamda])
angles_rad = [arcsin(x) for x in frequency*lamda]
angles_deg = [angle_rad/pi*180 for angle_rad in angles_rad]
parameters = [lamda,n.reshape(3),frequency,angles_rad, angles_deg]

height = 4e-6 #in microns
inc_angle = -25/180*pi
for period in range(30,100,4):
    period = period*1e-7
    blaze_angle = arctan(height/period)
    trans_angle = arcsin(mean(n)*sin(blaze_angle - arcsin(sin(-inc_angle+blaze_angle)/mean(n))))/pi*180
    print("Transmission maxima: "+str(trans_angle))
    display = []
    for i in range(len(parameters[0])):
        several_slits = transmissive(parameters[2][i], parameters[1][i], parameters[0][i], inc_angle = inc_angle)
        display.append([parameters[4][i],several_slits])
    fig = plt.figure(figsize=(30,6))
    ax = fig.add_subplot(1, 1, 1)
    plt.title("Transmissive", fontsize = 24)
    plt.text(0.02, 0.8, 'Period: '+str(round(period,8))+'m \nBlaze angle: '+str(round(blaze_angle/pi*180,1)),transform = ax.transAxes, fontsize =24)
    plt.plot(display[0][0], display[0][1], color='blue', label = '470 nm')
    plt.plot(display[1][0], display[1][1], color='green', label = '530 nm')
    plt.plot(display[2][0], display[2][1], color='red', label = '680 nm')
    plt.legend(fontsize=24)
    plt.vlines(trans_angle, 0,max(max(display[0][1]), max(display[1][1]), max(display[2][1])))
    plt.xlabel('Difraction angle[°]')
    plt.ylabel('Intesity')
    plt.show()
    
    ref_angle = -inc_angle - 2*blaze_angle
    print("Reflection maxima: "+str(ref_angle/pi*180))
    display = []
    for i in range(len(parameters[0])):
        several_slits = reflective(parameters[2][i], parameters[1][i], parameters[0][i], inc_angle = inc_angle)
        display.append([parameters[4][i],several_slits])
    fig = plt.figure(figsize=(30,6))
    ax = fig.add_subplot(1, 1, 1)
    plt.title("Reflective", fontsize = 24)
    plt.text(0.02, 0.8, 'Period: '+str(round(period,8))+'m \nBlaze angle: '+str(round(blaze_angle/pi*180,1)),transform = ax.transAxes, fontsize =24)
    plt.plot(display[0][0], display[0][1], color='blue', label = '470 nm')
    plt.plot(display[1][0], display[1][1], color='green', label = '530 nm')
    plt.plot(display[2][0], display[2][1], color='red', label = '680 nm')
    plt.legend(fontsize=24)
    plt.vlines(ref_angle/pi*180, 0,max(max(display[0][1]), max(display[1][1]), max(display[2][1])))
    plt.xlabel('Difraction angle[°]')
    plt.ylabel('Intesity')
    plt.show()

In [ ]:
plt.savefig('p:'+str(period)+'_ba:'+str(round(blaze_angle))+'_h:'+str(height)+'.jpg')